# Browser-Use

In [2]:
# !pip install google-genai
# !pip install nest_asyncio asyncio

# !pip install langchain langchain_openai langchain_google_genai langchain-core
# !pip install -U pip
# !pip install browser_use==0.1.41
# !pip install -U langchain_google_genai
# !pip install gradio
# !pip install dotenv
# !pip install langchain-community


# !pip install "anyio<4"

# !pip install playwright
# !playwright install

In [3]:
# !pip install -U langgraph langchain_google_genai langchain_community langgraph-checkpoint-postgres  openai langchain_groq

In [4]:
# !playwright install

In [5]:
# !playwright install chromium
# !playwright install-deps chromium

# !patchright install chromium

In [6]:
import asyncio
import nest_asyncio

from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from browser_use import Agent, Browser, BrowserConfig
import os
from google import genai
from pydantic import SecretStr
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
import requests
from openai import OpenAI


nest_asyncio.apply()   
load_dotenv(override=True)

home_dir = os.path.expanduser("~")
save_data_path = f'{home_dir}/Google Drive/My Drive/Projects/Data/Knowledge-base/Unsorted'


INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [telemetry] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.


In [7]:
use_which_llm = 'google'

#Config for local usage
insturction_llm_base_url = 'http://127.0.0.1:1234/v1'
insturction_model = 'qwen3-14b-mlx'
insturction_reporting_model = 'qwen2.5-14b-instruct-mlx'
insturction_api_key = 'None'

#Config for google LLM
google_api_key = os.getenv('GOOGLE_API_KEY')
google_data_gathering_model = 'gemini-2.0-flash' #Light weight model to gather data
google_reporting_model = 'gemini-2.0-flash' #Can chose a different model for reporting
planner_model =''
# model = "gemini-2.0-flash-lite"
# model = 'gemini-2.5-flash-preview-04-17'

# Config for agents
agent_max_steps = 100 
use_planner_model = False
generate_gif = False

# config browser
headless_mode = False
disable_security = False

initial_actions = []

main_task = 'crewai python'

sub_task = """
            research and cross-check infomation and fact-check from other sources,
            and get sources in URL Link and include them in the report.
            
            Include very detailed :
                Detailed Summary
                Detailed features
                Detailed Description
                URL link to the sources
                Reviews
                Detailed Pros and Cons
                use case
            """

task = f"""

        Deep reseach for: {main_task}

        {sub_task}

        Exclude writing:
            Save and share your research.

        """

report_contents = f"""
                Write me a detailed report on 
                {main_task} 
          
                also include in the report:    
                    {sub_task}
                
                Be as detailed as possible, 

                it is important to only Based on only the extracted data below:
            """

local_system_prompt_task = f""" 
                            Analyst the text and breakdown into steps 
                            following this rules

                            Structure
                                Be specific about the goal: Clearly state what you want to accomplish
                                Break complex tasks into steps: Use numbered lists for multi-step processes
                                Include format specifications: Mention desired output formats and structures
                                Provide context: Include relevant background information the agent needs
                                Set constraints: Specify any limitations or requirements the agent should follow
                            
                            Clarity
                                Use precise language: Avoid ambiguity and vague instructions
                                Specify parameters: Include exact values, ranges, or criteria
                                Clarify expectations: Explain what success looks like
                                Include examples: Provide examples of desired outputs when helpful
                                Define terminology: Explain industry-specific or technical terms
                            
                            Adaptability
                                Include fallback options: Provide alternatives if the primary approach fails
                                Specify error handling: Explain how to deal with potential issues
                                Allow for iteration: Build in opportunities for refinement
                                Request verification: Ask the agent to verify critical information
                                Include decision points: Define how choices should be made when alternatives exist
                            
                            Efficiency
                                Prioritize tasks: Indicate which aspects are most important
                                Set time or resource limits: Specify constraints on depth or breadth
                                Focus on value-add activities: Emphasize where the agent can provide the most value
                                Minimize unnecessary steps: Avoid redundant or low-value activities
                                Batch similar operations: Group related tasks for efficiency

                            Return in markdown
                            """

local_prompt_task = """
                    Analyst the text and Breakdown the task in simple details
                    
                    It is very critical that you answer only in a Step-by-Step instruction.
                    
                    Important to avoid sites with CAPTCHA.
                    
                    Don't include any other verbose explanatiouns, 
                    do not include website in the instruction.
                    and don't include the markdown syntax anywhere 
                    
                    and be very detailed:
                    """

local_prompt_task += task




In [8]:
def get_prompt_instruction(local_prompt_task, local_system_prompt_task, model, base_url, api_key):
    openai = OpenAI(base_url=base_url, api_key=api_key)
    
    messages = [
    {"role": "system",
     "content": local_system_prompt_task,
    "role": "user",
     "content": local_prompt_task}]

    response = openai.chat.completions.create(
            model = model,
            messages = messages,
            temperature = 0.0
        )
    
    return (response.choices[0].message.content).split('/think>')[-1]

In [9]:
instruction_prompt_gather_data = get_prompt_instruction(local_prompt_task, 
                                            local_system_prompt_task, 
                                            model=insturction_model, 
                                            base_url=insturction_llm_base_url, 
                                            api_key=insturction_api_key)
print(instruction_prompt_gather_data)



Step-by-Step instruction:  

1. Search for "crewai python" on Google and GitHub to identify primary sources.  
2. Verify the official documentation or repository for crewai python (e.g., GitHub page).  
3. Extract a detailed summary of crewai python, including its purpose and key functionalities.  
4. List all core features such as task delegation, agent roles, LLM integration (e.g., OpenAI), and scalability.  
5. Provide a detailed description of how crewai python operates, including agent collaboration and workflow management.  
6. Collect URL links from credible sources (e.g., GitHub, Medium articles, or technical blogs) that validate the information.  
7. Gather reviews from user forums, GitHub issues, or community platforms to assess reliability and performance.  
8. Compile a list of pros (e.g., ease of use, flexibility) and cons (e.g., limited documentation, dependency on LLMs).  
9. Identify use cases such as automating research tasks, customer service bots, or data analysis 

In [10]:
async def use_browser_to_get_data(task = instruction_prompt_gather_data):
    # Initialize the agent with the task and language model
    config = BrowserConfig(
                	headless=headless_mode, 
                    browser_binary_path= '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome',
                    extra_browser_args=[f"--user-data-dir={home_dir}/Library/Application Support/Google/Chrome/Default",]
                	# disable_security=True  # Uncomment if you want to disable security
                )
    
    browser = Browser(config=config)
    agent = Agent(
        task=task,
        llm=llm,  # Replace with your LLM configuration
        browser=browser,
        generate_gif=generate_gif, # Disable GIF generation
        use_vision = False,
        initial_actions = initial_actions,
        enable_memory=True,
        retry_delay = 60, 
        max_actions_per_step = 10,
        # validate_output=True
    )
    
    return await agent.run(max_steps = agent_max_steps)
  

match use_which_llm:
    case 'local':
        llm = ChatOpenAI(model=local_data_gathering_model, 
                                  base_url=local_llm_base_url, 
                                  api_key = 'None',
                                  temperature=0.0)
        
        os.environ['OPENAI_API_KEY'] = 'None'
        
    case 'google':
        llm = ChatGoogleGenerativeAI(model = google_data_gathering_model,
                               api_key = SecretStr(google_api_key),
                                     temperature=0.0)
        os.environ['GEMINI_API_KEY'] = google_api_key
    
    # Run the agent and get results asynchronously
result = asyncio.run(use_browser_to_get_data()) 
    
    
# for action in result.action_results():
#     print(action.extracted_content, end='\r', flush=True)

INFO     [agent] 🧠 Starting an agent with main_model=models/gemini-2.0-flash +memory, planner_model=None, extraction_model=None 


/opt/anaconda3/lib/python3.11/site-packages/browser_use/agent/service.py:321: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/opt/anaconda3/lib/python3.11/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/opt/anaconda3/lib/python3.11/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('zope')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declar

INFO     [datasets] PyTorch version 2.6.0 available.
INFO     [datasets] Polars version 1.27.1 available.
INFO     [datasets] TensorFlow version 2.16.2 available.
INFO     [faiss.loader] Loading faiss.
INFO     [faiss.loader] Successfully loaded faiss.
INFO     [faiss] Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
INFO     [mem0.vector_stores.faiss] Loaded FAISS index from /tmp/faiss/mem0.faiss with 3 vectors


<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


INFO     [agent] 🚀 Starting task: 

Step-by-Step instruction:  

1. Search for "crewai python" on Google and GitHub to identify primary sources.  
2. Verify the official documentation or repository for crewai python (e.g., GitHub page).  
3. Extract a detailed summary of crewai python, including its purpose and key functionalities.  
4. List all core features such as task delegation, agent roles, LLM integration (e.g., OpenAI), and scalability.  
5. Provide a detailed description of how crewai python operates, including agent collaboration and workflow management.  
6. Collect URL links from credible sources (e.g., GitHub, Medium articles, or technical blogs) that validate the information.  
7. Gather reviews from user forums, GitHub issues, or community platforms to assess reliability and performance.  
8. Compile a list of pros (e.g., ease of use, flexibility) and cons (e.g., limited documentation, dependency on LLMs).  
9. Identify use cases such as automating research tasks, custom

I0000 00:00:1747274997.118991  657227 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1747274997.516471  657227 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


INFO     [agent] 🤷 Eval: Unknown - Starting the task by searching for "crewai python" on Google and GitHub.
INFO     [agent] 🧠 Memory: Starting the task. Need to search Google and GitHub for "crewai python" to identify primary sources. Then, verify the official documentation or repository for crewai python (e.g., GitHub page). Extract a detailed summary of crewai python, including its purpose and key functionalities. List all core features such as task delegation, agent roles, LLM integration (e.g., OpenAI), and scalability. Provide a detailed description of how crewai python operates, including agent collaboration and workflow management. Collect URL links from credible sources (e.g., GitHub, Medium articles, or technical blogs) that validate the information. Gather reviews from user forums, GitHub issues, or community platforms to assess reliability and performance. Compile a list of pros (e.g., ease of use, flexibility) and cons (e.g., limited documentation, dependency on LLMs). Ide

/opt/anaconda3/lib/python3.11/site-packages/browser_use/browser/browser.py:375: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  clients = [obj for obj in gc.get_objects() if isinstance(obj, httpx.AsyncClient)]


In [11]:
for action in result.action_results():
    if action.extracted_content != None:
        report_contents += action.extracted_content
        report_contents += '\n\n'

client = genai.Client(api_key=google_api_key)

response = client.models.generate_content(
            model=google_reporting_model, 
            contents=report_contents)


INFO     [google_genai.models] AFC is enabled with max remote calls: 10.
INFO     [google_genai.models] AFC remote call 1 is done.


In [12]:
Markdown(response.text.replace('```markdown', '').replace('```', ''))

## CrewAI Python: Detailed Report

This report provides a comprehensive overview of CrewAI, a Python framework for building collaborative AI agents, based on the extracted data. It includes a summary, features, description, sources, reviews, pros & cons, and use cases.

**Detailed Summary:**

CrewAI is a Python framework designed to empower developers in creating autonomous AI agent teams that collaboratively solve complex problems. Unlike other agent frameworks, CrewAI has been built from scratch, independent of frameworks like LangChain, offering a unique balance of high-level simplicity and low-level control. It allows developers to define agent roles, assign them specific tools, and manage their interactions to achieve complex objectives. CrewAI operates in two primary modes: "Crews," focusing on collaborative intelligence with role-based agents, and "Flows," offering structured automation with granular workflow control. It aims to provide a production-ready, scalable, and secure solution for enterprise-level AI automation.

**Detailed Features:**

CrewAI provides a robust set of features for building and managing AI agent teams:

*   **Task Delegation:** AI Agents can delegate tasks amongst each other, enabling a collaborative and efficient workflow.
*   **Agent Roles:** Define specific roles for each agent, including their expertise, goals (e.g., researcher, analyst, writer), and tools they utilize.
*   **LLM Integration:**  Connects to any Large Language Model (LLM), allowing for flexible integration with various models based on your specific needs and resources.
*   **Scalability:**  Built for reliability and scalability in real-world applications, making it production-ready for handling complex, enterprise-level tasks.
*   **Workflow Management:** Offers two workflow management systems:
    *   **Crews:** Manages AI agent teams, oversees workflows, and ensures seamless collaboration between agents.
    *   **Flows:** Provides structured automation with granular control over workflow execution, including conditional logic, loops, and dynamic state management.
*   **Intelligent Collaboration:**  Agents work together, sharing insights and coordinating tasks to achieve complex objectives. The Process component ensures smooth collaboration.
*   **Event-Driven Orchestration:** CrewAI allows for event-driven orchestration, enabling AI systems to respond dynamically to changing conditions.
*   **State Management:** Support state management, allowing for tracking progress and managing complex workflows.
*   **Error Handling:** Include robust error handling mechanisms for building reliable AI systems.

**Detailed Description:**

CrewAI allows developers to define AI agents with specific roles, tools, and objectives. These agents can then work together to solve complex problems through a process of task delegation and collaboration. CrewAI supports two primary modes of operation: Crews and Flows.

*   **Crews:** Crews are designed for scenarios where collaboration and intelligence are paramount. Developers can create teams of AI agents, each with defined roles, tools, and goals.  The framework handles the orchestration of these agents, ensuring that they work together effectively to achieve a common objective. This mode is optimized for autonomy and collaborative intelligence.

*   **Flows:** Flows are designed for scenarios where structured automation and granular control are required.  Developers can define workflows with conditional logic, loops, and dynamic state management. This mode provides precise control over workflow execution, ensuring tasks are executed reliably and efficiently. Flows are ideal for tasks that require deterministic execution and predictable outcomes.

CrewAI is also becoming a standard for enterprise-ready AI automation, with a large community of certified developers.

**URL Links to Sources:**

*   **CrewAI Documentation (Introduction):** [https://docs.crewai.com/introduction](https://docs.crewai.com/introduction)
*   **CrewAI GitHub Repository:**  [https://github.com/crewai/crewai](This link was initially a "Page not found" error, suggesting the repository name or link may have changed.)

**Reviews:**

*   **Community Courses:** "With over 100,000 developers certified through our community courses, CrewAI is rapidly becoming the standard for enterprise-ready AI automation."

**Detailed Pros and Cons:**

**Pros:**

*   **High-level Simplicity:** Easy to get started with and build basic AI agent teams.
*   **Precise Low-level Control:** Offers granular control over workflow execution and agent behavior.
*   **Optimized for Autonomy:** Designed to create autonomous agents that can operate independently.
*   **Collaborative Intelligence:**  Facilitates collaborative problem-solving between agents.
*   **Granular, Event-Driven Control:** Provides granular control over events within the CrewAI environment.
*   **Single LLM Calls:** Orchestrates tasks with precise control using individual LLM calls.
*   **Native Crew Integration:** Seamlessly supports the Crews workflow management system.
*   **Role-Based Agents:** Allows for the definition of agents with specific roles and expertise.
*   **Flexible Tools:** Supports the integration of custom tools and APIs.
*   **Intelligent Collaboration:** Agents work together, sharing insights and coordinating tasks.
*   **Task Management:** Offers robust task management capabilities.
*   **Event-Driven Orchestration:** Enables event-driven workflows and responses.
*   **Fine-Grained Control:** Provides fine-grained control over various aspects of the AI system.
*   **Deterministic Execution:** Supports deterministic execution for predictable outcomes.
*   **Autonomous Operation:** Enables autonomous operation of AI agent teams.
*   **Natural Interaction:** Facilitates natural interaction between agents and the environment.
*   **Extensible Design:** Designed with extensibility in mind, allowing for customization.
*   **Production Ready:** Built for reliability and scalability in real-world applications.
*   **Security-Focused:** Emphasizes security in its design and operation.
*   **Cost-Efficient:** Designed to be cost-efficient in terms of LLM usage.

**Cons:**

*   **Dependency on LLMs:**  Reliance on Large Language Models (LLMs) for agent intelligence, functionality, and task execution. This means performance is directly tied to the quality and availability of the chosen LLM.

**Use Cases:**

*   **Automating Research Tasks (Open-Ended Research):**  Automate the process of gathering and synthesizing information from various sources.
*   **Content Generation (Articles, Reports, Marketing Materials):** Generate high-quality content for various purposes, such as articles, reports, and marketing materials.
*   **Decision Workflows (Predictable, Auditable Decision Paths):** Create predictable and auditable decision-making processes.
*   **API Orchestration (Reliable Integration with Multiple External Services in a Specific Sequence):** Integrate with multiple external services in a reliable and specific sequence.
*   **Customer Service Bots:** Develop customer service bots that can provide natural and autonomous interaction.
*   **Data Analysis Workflows:**  Streamline data analysis workflows by assigning specific roles to agents and utilizing flexible tools.


In [13]:
counter = 0

file_name = ''
while (len(file_name) > 80) | (file_name == ''):
    file_name = response.text.replace('```markdown', '')\
      .replace('```', '').splitlines()[counter]\
    .replace('#', '')\
    .replace(':', '')\
    .replace('*', '')
    
    counter+= 1
    
f = open(f'{save_data_path}/{file_name}_insturction_{insturction_model}_gathering_{google_data_gathering_model}_report{google_reporting_model}.txt', 'w')
f.write(response.text.replace('```markdown', '').replace('```', ''))
f.close()